In [1]:
# 2. Instalar versión compatible
!pip install numpy==1.23.5


In [2]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 1.6 MB/s eta 0:00:00


In [3]:
import h2o
from h2o.automl import H2OAutoML

# Iniciar el entorno de H2O
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.27" 2025-04-15; OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpeyv5cbt3
  JVM stdout: /tmp/tmpeyv5cbt3/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpeyv5cbt3/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,2 months and 3 days
H2O_cluster_name:,H2O_from_python_unknownUser_51ulgk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [5]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd

# Cargar dataset limpio
ruta = '/content/drive/MyDrive/Prediccion de Diabetes - Proyecto Analitica/proyecto-diabetes-analitica/Datos/dataset_limpio.csv'
df = pd.read_csv(ruta)

# Convertir a formato H2O
df_h2o = h2o.H2OFrame(df)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
# Definir Variable objetivo
y = 'diabetes'

# Variables predictoras (todas menos la de salida)
X = [col for col in df.columns if col != y]

# Asegurar que la variable objetivo sea categórica
df_h2o[y] = df_h2o[y].asfactor()


In [8]:
#División del dataset
train, test = df_h2o.split_frame(ratios=[0.8], seed=42)


In [9]:
# Entrenamiento de modelos con H2O AutoML
aml = H2OAutoML(max_runtime_secs=300, seed=42, balance_classes=True)
aml.train(x=X, y=y, training_frame=train)


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/3
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [10]:
# Evaluar el mejor modelo
# Mostrar leaderboard (mejores modelos)
lb = aml.leaderboard
lb.head()

# Evaluar en el test set
perf = aml.leader.model_performance(test_data=test)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.02198549445344182
RMSE: 0.14827506349161285
LogLoss: 0.08098442799082718
AUC: 0.9784260152484804
AUCPR: 0.8825893931959331
Gini: 0.9568520304969608
Null degrees of freedom: 20032
Residual degrees of freedom: 20029
Null deviance: 11378.253240142136
Residual deviance: 3244.7220918804824
AIC: 3252.7220918804824

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5442135208109476
       0      1     Error    Rate
-----  -----  ----  -------  ---------------
0      18357  31    0.0017   (31.0/18388.0)
1      487    1158  0.296    (487.0/1645.0)
Total  18844  1189  0.0259   (518.0/20033.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.544214     0.817219  102
max f2                       0.143931     0.783113  244
max f0point5                 0.787215     0.91587   78
max accuracy                 0.693838     0.974292  84
max precision                0.995172     1         0
max recall                   0.00331399   1         382
max specificity              0.995172     1         0
max absolute_mcc             0.693838     0.817329  84
max min_per_class_accuracy   0.103614     0.91003   271
max mean_per_class_accuracy  0.0939349    0.911868  278
max tns                      0.995172     18388     0
max fns                      0.995172     1623      0
max fps                      0.000231142  18388     399
max tps                      0.00331399   1645      382
max tnr                      0.995172     1         0
max fnr                      0.995172     0.986626  0
max fpr                      0.000231142  1         399
max tpr                      0.00331399   1         382

Gains/Lift Table: Avg response rate:  8.21 %, avg score:  8.57 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100334                   0.983487           12.1781    12.1781            1                0.989656     1                           0.989656            0.122188        0.122188                   1117.81   1117.81            0.122188
2        0.020017                    0.970343           12.1781    12.1781            1                0.976709     1                           0.983199            0.121581        0.243769                   1117.81   1117.81            0.243769
3        0.0300005                   0.958211           12.1781    12.1781            1                0.964594     1                           0.977007            0.121581        0.36535                    1117.81   1117.81            0.36535
4        0.0400339                   0.943883           12.1781    12.1781            1                0.951505     1                           0.970616            0.122188        0.487538                   1117.81   1117.81            0.487538
5        0.0500175                   0.915945           12.1781    12.1781            1                0.931815     1                           0.962871            0.121581        0.609119                   1117.81   1117.81            0.609119
6        0.100035                    0.205251           4.04722    8.11267            0.332335         0.399744     0.666168                    0.681308            0.202432        0.81155                    304.722   711.267            0.775168
7        0.150002                    0.110349           1.83706    6.02219            0.150849         0.151

In [11]:
# guardar el modelo
modelo_guardado = h2o.save_model(model=aml.leader, path="/content/drive/MyDrive/Prediccion de Diabetes - Proyecto Analitica/proyecto-diabetes-analitica/Codigos", force=True)
print("Modelo guardado en:", modelo_guardado)


Modelo guardado en: /content/drive/MyDrive/Prediccion de Diabetes - Proyecto Analitica/proyecto-diabetes-analitica/Codigos/StackedEnsemble_BestOfFamily_1_AutoML_1_20250531_102955
